# Linear Solver Performance: Evaluation of Single Core Benchmarks

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now); 

In [ ]:
//#r "C:\Users\jenkinsci\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Application.SipPoisson;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Manually add database (e.g. when accessing the database of user `JenkinsCI` as some other, regular guy)
//var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\bkup-2022Jul16_074633.WIP-k-LinslvPerfSer");

In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfSer") ?? "WIP-k-LinslvPerfSer"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

In [ ]:
//databases = new [] { tempDb };

In [ ]:
databases

In [ ]:
//foreach(var s in wmg.Sessions) {
//    Console.WriteLine(s.Database.Path);
//}

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.SuccessfulTermination == false
                                              || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0);
FailedSessions

In [ ]:
//NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not terminate successfully.");

In [ ]:
//FailedSessions.ForEach(s => s.Delete(true));

## Create Table for Post-Processing

The timing information that is requires for this study is not present in the default session table.
Instead, it must be extracted from the *session profiling*.

To obtain timing-measurements, the instrumentation generated by certain `BlockTrace`-blocks 
within the `UniSolver`, resp. the `XdgBDFTimestepping` class is extracted and added to the data table:

In [ ]:
// evaluators to add additional columns to the session table
static class AddCols {
    static int GetSolType(ISessionInfo SI) {
        if(SI.GetControl().GetSolverType().Name.Contains("XNSE"))
            return 3;
        if(SI.GetControl().GetSolverType().Name.Contains("XdgPoisson3Main"))
            return 2;
        return 1;
    }
    
    
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        ilPSP.Tracing.MethodCallRecord nd;
        if(GetSolType(SI) == 3)
            nd = mcr.FindChildren("*.XdgTimestepping.ComputeOperatorMatrix").Single();
        else 
            nd  = mcr.FindChildren("MatrixAssembly").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Aggregation_basis_init").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Init").Single();
        //var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Run").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
    }
    static public object ComputeNodeName(ISessionInfo SI) {
        return SI.ComputeNodeNames.First();
    }
    
    //static public object DOFs(ISessionInfo SI) {
    //    int NoOfItr = Convert.ToInt32(SI.KeysAndQueries["NoIter"]);
    //    return NoOfItr;
    //}
    
    static public object DGdegree(ISessionInfo SI) {
        switch(GetSolType(SI)) {
            case 3:
            return SI.KeysAndQueries.Single(kv => kv.Key.Contains("DGdegree:Velocity")).Value;
            case 2:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:u").Value;    
            case 1:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:T").Value;
        }
        throw new ArgumentException();
    }
}

In [ ]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
wmg.AdditionalSessionTableColums.Add("ComputeNodeName", AddCols.ComputeNodeName);
wmg.AdditionalSessionTableColums.Add("DGdegree", AddCols.DGdegree);
//wmg.AdditionalSessionTableColums.Add("DOFs", AddCols.DOFs);

In [ ]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

List of all available Data Columns:

In [ ]:
SessTab.GetColumnNames().ToConcatString("", "; ", "")

Select those columns which are of interest:

In [ ]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree", "Grid:NoOfCells", "LinearSolver.Name", "LinearSolver.Shortname", "DOFs", "MatrixAssembly",
    "Grid:SpatialDimension", "NoOfCores",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter", "ComputeNodeName", "RegularTerminated");

In [ ]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

Only consider runs which have been successful:

In [ ]:
SubTab = SubTab.ExtractRows(delegate(int iRow, IDictionary<string, object> row) {
    return (bool)(row["RegularTerminated"]);
});

In [ ]:
//SubTab = SubTab.ExtractRows(delegate(int iRow, IDictionary<string, object> row) {
//    return (bool)(row["RegularTerminated"]) && ((string)row["SessionName"]).Contains("Poisson");
//});

In [ ]:
//SubTab.Print();

## Vizualisation of Results

The following data is available:

In [ ]:
SubTab.GetColumnNames()

Available DG degrees:

In [ ]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree").ToSet().OrderBy(s => s).ToArray();
DGdegrees

All used solvers:

In [ ]:
SubTab.GetColumn<string>("LinearSolver.Shortname").ToSet()

In [ ]:
const string Pardiso = "PARDISO";
const string OrmgSwz = "OrthoMG w Add Swz";
const string GmrsP2g = "GMRES w p2G";
const string OrmgPmg = "Ortho w pmG";
const string OrmgILU = "OrthoMG w ILU";

Cases investigated:

In [ ]:
const string Poisson = "SIP_Poisson";
const string XPoisson = "XdgPoisson";
const string Stokes2D = "BottiPietroStokes2D";
const string Stokes3D = "BottiPietroStokes3D";
const string XStokes = "XdgStokes";
string[] AllCases = new string[] { Poisson, XPoisson, Stokes2D, Stokes3D, XStokes };

In [ ]:
string ExtractCase(string sessionName) {
    return AllCases.Single(caseName => sessionName.Contains(caseName));
}

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [ ]:
PlotFormat SlvCode2Pltfmt(string solver_name, string caseName) { 
    var Fmt = new PlotFormat();
    switch(solver_name) {
        case Pardiso: 
            Fmt.PointType = PointTypes.Asterisk;
            Fmt.DashType  = DashTypes.Dotted;
            break;
        case GmrsP2g: 
            Fmt.PointType = PointTypes.OpenUpperTriangle;
            break;
        case OrmgSwz: 
            Fmt.PointType = PointTypes.Box;
            break;
        case OrmgILU:
            Fmt.PointType = PointTypes.Diamond;
            break;
        case OrmgPmg:
            Fmt.PointType = PointTypes.OpenLowerTriangle;
            break;
        default:
            Console.WriteLine("unknown: " + solver_name); 
            break;
    } 
    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.85;
    Fmt.LineWidth = 2;    
    Fmt.Style     = Styles.LinesPoints;
    
    if(caseName.Contains(Poisson))
        Fmt.LineColor = LineColors.Blue;
    else if(caseName.Contains(XPoisson))
        Fmt.LineColor = LineColors.Red;
    else if(caseName.Contains(Stokes2D))
        Fmt.LineColor = LineColors.Green;
    else if(caseName.Contains(Stokes3D))
        Fmt.LineColor = LineColors.Blue;    
    else if(caseName.Contains(XStokes))
        Fmt.LineColor = LineColors.Red;
    

    return Fmt;
}

The following function will later be used to check the regression solpe of runtimes
(ideally, one expects linear runtime of a solver with respect to degrees-of-freedom):

In [ ]:
void AssertSlope(Plot2Ddata plot, string caseName, string solverName, double allowedSlope, string info) {
   NUnit.Framework.Assert.LessOrEqual(
      plot.Regression().Single(tt =>  tt.Key.Contains(caseName) && tt.Key.Contains(solverName)).Value,
      allowedSlope,
      $"{solverName}/{info} scaling for {caseName} exceeds limit");
}

In [ ]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax, double LegendYpos) {

int rows    = DGdegrees.Length;
int columns = 2;
string[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows + 1,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    //if(pDegree > rows*columns-1)
    //    continue;
    //int tmpDG = -1;
    //if(pDegree < DGdegrees.Length)
    //    tmpDG = DGdegrees[pDegree];
    
    int tmpDG = DGdegrees[iRow];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           string solver_name = (string) (Row["LinearSolver.Shortname"]);
           //ignore the solvers specified in ingore_solvers
           foreach(string sc in ignore_solvers){
               if(solver_name == sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           string caseName = ExtractCase( (string) Row["SessionName"]);
           if(iCol == 0) {
               // in Column 0, draw only Poisson
               if(!caseName.Contains("Poisson")) {
                   Nmn = null;
                   Fmt = null;
                   return;
               }
           } else if(iCol == 1) {
               // in Column 1, draw only Poisson
               if(!caseName.Contains("Stokes")) {
                   Nmn = null;
                   Fmt = null;
                   return;
               }
           } else {
               throw new NotImplementedException();
           }
           
           
           //process the other solvers
           Fmt = SlvCode2Pltfmt(solver_name, caseName);
           Nmn = solver_name + "/" + caseName;
       });

       // plot the linear behavior reference line
       double[] dof = new[] { 1e3, 1e6 }; // x-limits of the reference-line-plot
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e7;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 5;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == rows - 1) {
          multiplots[iRow,iCol].ShowLegend = true;
          //multiplots[iRow,iCol].LegendAlignment = new string[]{"o", "r", "t" };
          multiplots[iRow,iCol].LegendFont = 12;
          multiplots[iRow,iCol].Legend_maxrows = 100;
          multiplots[iRow,iCol].LegendPosition = new double[] { 4e7, LegendYpos };
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

### Total Runtime

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [ ]:
var multiplotsRtime = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4, 1e-5);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplotsRtime.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

In [ ]:
//var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4, 1e-5);
//multiplots.PlotCairolatex().SaveTo("Runtime.tex");

In [ ]:
/*var legend = new Plot2Ddata();
legend.AddDataGroup("Pardiso",new double[] {1,2},new double[] {1,2}, "-og");
legend.ShowXtics = false;
legend.ShowXtics = false;
//legend.XrangeMin = 3;
//legend.XrangeMax = 3.1;

//legend.AddGnuplotCommand("set border 0");
//legend.AddGnuplotCommand("set style line 101 lc rgb '#808080' lt 1 lw 1");
legend.AddGnuplotCommand("unset xlabel");
legend.AddGnuplotCommand("unset ylabel");
legend.AddGnuplotCommand("set format x ''");
legend.AddGnuplotCommand("set format y ''");
legend.AddGnuplotCommand("set tics scale 0");

legend.PlotNow()*/

#### Regression and Error checking (total runtime)

For polynomial degree 2:

In [ ]:
multiplotsRtime[0,0].Regression()

In [ ]:
multiplotsRtime[0,1].Regression()

For polynomial degree 3:

In [ ]:
multiplotsRtime[1,0].Regression()

In [ ]:
multiplotsRtime[1,1].Regression()

For polynomial degree 5:

In [ ]:
multiplotsRtime[2,0].Regression()

In [ ]:
multiplotsRtime[2,1].Regression()

In [ ]:
AssertSlope(multiplotsRtime[0,0], Poisson,  GmrsP2g, 1.14, "k=2");
AssertSlope(multiplotsRtime[0,0], XPoisson, GmrsP2g, 1.0,  "k=2");
AssertSlope(multiplotsRtime[0,1], Stokes2D, GmrsP2g, 1.0,  "k=2");
AssertSlope(multiplotsRtime[0,1], XStokes,  GmrsP2g, 1.5,  "k=2");

AssertSlope(multiplotsRtime[0,0], Poisson,  OrmgSwz, 1.2,  "k=2");
AssertSlope(multiplotsRtime[0,0], XPoisson, OrmgSwz, 1.0,  "k=2");
AssertSlope(multiplotsRtime[0,1], Stokes2D, OrmgSwz, 1.2,  "k=2");
AssertSlope(multiplotsRtime[0,1], XStokes,  OrmgSwz, 1.7,  "k=2"); // really bad!

AssertSlope(multiplotsRtime[0,0], Poisson,  OrmgPmg, 1.1,  "k=2");
AssertSlope(multiplotsRtime[0,0], XPoisson, OrmgPmg, 1.1, "k=2");
//AssertSlope(multiplotsRtime[0,1], Stokes2D, OrmgPmg, 1.4, "k=2");
//AssertSlope(multiplotsRtime[0,1], XStokes,  OrmgPmg, 1.0,  "k=2"); // currently NAN

In [ ]:
AssertSlope(multiplotsRtime[1,0], Poisson, GmrsP2g, 1.11, "k=3");
AssertSlope(multiplotsRtime[0,0], XPoisson, GmrsP2g, 1.0,  "k=3");
AssertSlope(multiplotsRtime[0,1], Stokes2D, GmrsP2g, 1.0,  "k=3");
AssertSlope(multiplotsRtime[0,1], XStokes,  GmrsP2g, 1.5,  "k=3");

AssertSlope(multiplotsRtime[0,0], Poisson,  OrmgSwz, 1.2,  "k=3");
AssertSlope(multiplotsRtime[0,0], XPoisson, OrmgSwz, 1.0,  "k=3");
AssertSlope(multiplotsRtime[0,1], Stokes2D, OrmgSwz, 1.2,  "k=3");
AssertSlope(multiplotsRtime[0,1], XStokes,  OrmgSwz, 1.7,  "k=3"); // really bad!

AssertSlope(multiplotsRtime[0,0], Poisson,  OrmgPmg, 1.1,  "k=3");
AssertSlope(multiplotsRtime[0,0], XPoisson, OrmgPmg, 1.1, "k=3");
//AssertSlope(multiplotsRtime[0,1], Stokes2D, OrmgPmg, 1.4, "k=3"); // not really good
//AssertSlope(multiplotsRtime[0,1], XStokes,  OrmgPmg, 1.0,  "k=3"); // currently NAN

In [ ]:
AssertSlope(multiplotsRtime[2,0], Poisson, GmrsP2g, 1.1, "k=5");
AssertSlope(multiplotsRtime[0,0], XPoisson, GmrsP2g, 1.0,  "k=5");
AssertSlope(multiplotsRtime[0,1], Stokes2D, GmrsP2g, 1.0,  "k=5");
//AssertSlope(multiplotsRtime[0,1], XStokes,  GmrsP2g, 1.4,  "k=5");

AssertSlope(multiplotsRtime[0,0], Poisson,  OrmgSwz, 1.1,  "k=5");
AssertSlope(multiplotsRtime[0,0], XPoisson, OrmgSwz, 1.0,  "k=5");
AssertSlope(multiplotsRtime[0,1], Stokes2D, OrmgSwz, 1.2,  "k=5");
//AssertSlope(multiplotsRtime[0,1], XStokes,  OrmgSwz, 1.7,  "k=5"); // currently NAN

AssertSlope(multiplotsRtime[0,0], Poisson,  OrmgPmg, 1.1, "k=5");
AssertSlope(multiplotsRtime[0,0], XPoisson, OrmgPmg, 1.1, "k=5");
//AssertSlope(multiplotsRtime[0,1], Stokes2D, OrmgPmg, 1.4, "k=5"); // not really good
//AssertSlope(multiplotsRtime[0,1], XStokes,  OrmgPmg, 1.0,  "k=5"); // currently NAN

### Number of Iterations

For the iterative solvers, obe would expect **almost constant number of solver iterations**, 
independent of grid resolution. It may, however depend in nonlinar fashion on the DG polynomial order.

In [ ]:
var multiplotsIter = PlotSolverBehave("NoIter", false, 0, 155, -50);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplotsIter.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

The following function will later be used to check the regression solpe of runtimes
(ideally, one expects constant number of iterations of a multigrid-solver with respect to degrees-of-freedom):

In [ ]:
void AssertLimit(Plot2Ddata plot, string caseName, string solverName, double allowedLimit, string info) {
    var max = plot.dataGroups.Single(tt =>  tt.Name.Contains(caseName) && tt.Name.Contains(solverName)).Values.Max();
    Console.WriteLine($"Maximum {solverName}/{info} for {caseName} is " + max);
    NUnit.Framework.Assert.LessOrEqual(
      max,
      allowedLimit,
      $"{solverName}/{info} maximum for {caseName} exceeds limit");
}


In [ ]:
multiplotsIter[0,0].dataGroups

In [ ]:
multiplotsIter[0,1].dataGroups

In [ ]:
AssertLimit(multiplotsIter[0,0], Poisson,  GmrsP2g, 130, "k=2");
AssertLimit(multiplotsIter[0,0], XPoisson, GmrsP2g, 70, "k=2");
AssertLimit(multiplotsIter[0,1], Stokes2D, GmrsP2g, 80, "k=2");

AssertLimit(multiplotsIter[0,0], Poisson,  OrmgSwz, 10, "k=2");
AssertLimit(multiplotsIter[0,0], XPoisson, OrmgSwz, 10, "k=2");
AssertLimit(multiplotsIter[0,1], Stokes2D, OrmgSwz, 25, "k=2");  // not working yet
AssertLimit(multiplotsIter[0,1], XStokes,  OrmgSwz, 70, "k=2"); // not working yet

AssertLimit(multiplotsIter[0,0], Poisson,  OrmgPmg, 35, "k=2");
AssertLimit(multiplotsIter[0,0], XPoisson, OrmgPmg, 60, "k=2"); // not really ideal
//AssertLimit(multiplotsIter[0,1], Stokes2D, OrmgPmg, 80, "k=2");  // not working yet
//AssertLimit(multiplotsIter[0,1], XStokes,  OrmgPmg, 130, "k=2"); // not working yet

AssertLimit(multiplotsIter[0,0], Poisson,  Pardiso, 1, "k=2");
AssertLimit(multiplotsIter[0,0], XPoisson, Pardiso, 1, "k=2");
AssertLimit(multiplotsIter[0,1], Stokes2D, Pardiso, 1, "k=2"); 
AssertLimit(multiplotsIter[0,1], XStokes,  Pardiso, 1, "k=2");

In [ ]:
multiplotsIter[1,0].dataGroups

In [ ]:
multiplotsIter[1,1].dataGroups

In [ ]:
AssertLimit(multiplotsIter[1,0], Poisson,  GmrsP2g, 150, "k=3");  // not really ideal
AssertLimit(multiplotsIter[1,0], XPoisson, GmrsP2g, 120, "k=3");
AssertLimit(multiplotsIter[1,1], Stokes2D, GmrsP2g, 230, "k=3"); // not really ideal

AssertLimit(multiplotsIter[1,0], Poisson,  OrmgSwz, 15, "k=3");
AssertLimit(multiplotsIter[1,0], XPoisson, OrmgSwz, 15, "k=3");
AssertLimit(multiplotsIter[1,1], Stokes2D, OrmgSwz, 15, "k=3");  // not working yet
AssertLimit(multiplotsIter[1,1], XStokes,  OrmgSwz, 70, "k=3"); // slightly improved

AssertLimit(multiplotsIter[1,0], Poisson,  OrmgPmg, 10, "k=3");
AssertLimit(multiplotsIter[1,0], XPoisson, OrmgPmg, 20, "k=3");
AssertLimit(multiplotsIter[1,1], Stokes2D, OrmgPmg, 25, "k=3");
//AssertLimit(multiplotsIter[1,1], XStokes,  OrmgPmg, 1, "k=3"); // not working

AssertLimit(multiplotsIter[1,0], Poisson,  Pardiso, 1, "k=3");
AssertLimit(multiplotsIter[1,0], XPoisson, Pardiso, 1, "k=3");
AssertLimit(multiplotsIter[1,1], Stokes2D, Pardiso, 1, "k=3"); 
AssertLimit(multiplotsIter[1,1], XStokes,  Pardiso, 1, "k=3");

In [ ]:
multiplotsIter[2,0].dataGroups

In [ ]:
multiplotsIter[2,1].dataGroups

In [ ]:
AssertLimit(multiplotsIter[2,0], Poisson,  GmrsP2g, 200, "k=5");
AssertLimit(multiplotsIter[2,0], XPoisson, GmrsP2g, 200, "k=5");
//AssertLimit(multiplotsIter[2,1], Stokes2D, GmrsP2g, 230, "k=5");  // not working yet

AssertLimit(multiplotsIter[2,0], Poisson,  OrmgSwz, 8, "k=5");
AssertLimit(multiplotsIter[2,0], XPoisson, OrmgSwz, 8, "k=5");
AssertLimit(multiplotsIter[2,1], Stokes2D, OrmgSwz, 15, "k=5"); 
AssertLimit(multiplotsIter[2,1], XStokes,  OrmgSwz, 20, "k=5"); // not working yet

AssertLimit(multiplotsIter[2,0], Poisson,  OrmgPmg, 10, "k=5");
AssertLimit(multiplotsIter[2,0], XPoisson, OrmgPmg, 49, "k=5");
AssertLimit(multiplotsIter[2,1], Stokes2D, OrmgPmg, 20, "k=5");
//AssertLimit(multiplotsIter[2,1], XStokes,  OrmgPmg, 1, "k=5"); // not working

AssertLimit(multiplotsIter[2,0], Poisson,  Pardiso, 1, "k=5");
AssertLimit(multiplotsIter[2,0], XPoisson, Pardiso, 1, "k=5");
AssertLimit(multiplotsIter[2,1], Stokes2D, Pardiso, 1, "k=5"); 
AssertLimit(multiplotsIter[2,1], XStokes,  Pardiso, 1, "k=5");

### Runtime per Iteration

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [ ]:
var multiplotsTimePerIter = PlotSolverBehave("Solver_TimePerIter", true, 1e-2, 1e+6, 1e-3);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplotsTimePerIter.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

In [ ]:
multiplotsTimePerIter[0,0].Regression()

In [ ]:
multiplotsTimePerIter[0,1].Regression()

In [ ]:
multiplotsTimePerIter[1,0].Regression()

In [ ]:
multiplotsTimePerIter[1,1].Regression()

In [ ]:
multiplotsTimePerIter[2,0].Regression()

In [ ]:
multiplotsTimePerIter[2,1].Regression()

In [ ]:
AssertSlope(multiplotsTimePerIter[0,0], Poisson, GmrsP2g, 1.1, "k=2");
AssertSlope(multiplotsTimePerIter[1,0], Poisson, GmrsP2g, 1.1, "k=3");
AssertSlope(multiplotsTimePerIter[2,0], Poisson, GmrsP2g, 1.1, "k=5");

## Appendix: Table Form of Extracted Data

In [ ]:
SubTab.Print()